In [0]:
from google.colab import drive

GOOGLE_DRIVE_MOUNT_DIR = "/content/drive"
drive.mount(GOOGLE_DRIVE_MOUNT_DIR, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import matplotlib.pyplot as plt
import pandas as pd


from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras import losses, optimizers, activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


#Model Customization

In [0]:
#change number of classes according your configuration
CLASSES = 4

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape = (200,200,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# training all the layers
for layer in base_model.layers:
  layer.trainable = True
      
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

74842112/74836368 [==============================] - 1s 0us/step


In [0]:
#visualize trainable layers
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

In [0]:
model.summary()

#Data preparation

In [0]:
WIDTH = 200
HEIGHT = 200
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale = 1./255)

val_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(HEIGHT, WIDTH),
		batch_size=BATCH_SIZE,
		class_mode='categorical',
    shuffle=True)
    
validation_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

train_generator.class_indices


#Initializing Training

In [0]:
EPOCHS = 20

filepath = "drive_path/model_name.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_list)

#Confusion Matrix

In [0]:
Y_pred = model2.predict(test_generator, batch_size=BATCH_SIZE, verbose=1, steps=len(test_generator), workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-13', '14-17', '18-35', '36-r']
print(classification_report(test_generator.classes, y_pred, digits=4, target_names=target_names))

#Plot History of training

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Reload Model and Training process

In [0]:
model2 = load_model('drive_path/model_name.hdf5')

for layer in model2.layers:
  layer.trainable = True

model2.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

In [0]:
model2.summary()

In [0]:
train_generator.reset()
validation_generator.reset()

In [0]:
EPOCHS = 10
BATCH_SIZE = 32

filepath = "drive_path/model_name.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]


history = model2.fit_generator(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks)

#save all  
NAME = "model-DenseNet-4-classes"  
model2.save(f"drive_path/{NAME}.model")  

#Making Predictions

Reset test_generator for other prediction

In [0]:
test_generator.reset()

In [0]:
HEIGHT = 200
WIDTH = 200
BATCH_SIZE = 32

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)



##### Making a DataFrame with the predictions

In [0]:
model2 = load_model('drive_path/model_name.hdf5')

pred = model2.predict_generator(test_generator, verbose=1, steps=len(test_generator))
predicted_class_indices = np.argmax(pred, axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames = test_generator.filenames
results = pd.DataFrame({"Filename":filenames,
                        "Predictions":predictions})

results
results.to_csv(r'drive_path/filename.csv')

#Reload best checkpoint and Confusion Matrix

In [0]:
model2 = load_model('drive_path/model_name.hdf5')

Y_pred = model2.predict(test_generator, batch_size=BATCH_SIZE, verbose=1, steps=len(test_generator), workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-13', '14-17', '18-35', '36-r']
print(classification_report(test_generator.classes, y_pred, digits=4, target_names=target_names))